In [ ]:
#import packages
import pandas as pd
import datetime as dt
import numpy as np

import os


In [ ]:
#missing datas / map empty cells
#=> terminal : pip install missingno
!python3 -m pip install missingno
import missingno as mno



In [ ]:
#import file and first infos
list_clics = pd.read_excel("C:/Users/BGE/ownCloud/Arnaud/DEVELOPPEMENT/ACTIVITE/Sponso owncloud perso AS/Conventions et suivi activité/SUIVI CLICS/2024/export_mises_en_relation_2024_03_29.xlsx")
list_clics.info()

In [ ]:
# discover structure of dataframe
list_clics.head()


In [ ]:
#nb of lines
list_clics.count()

In [ ]:
# column structures
list_clics.columns

In [ ]:
# duplicates
list_unduplicated = list_clics.sort_values(by=["Entrepreneur", "Partenaire"])
list_unduplicated.drop_duplicates()
list_unduplicated.count()

In [ ]:
#truncate without hours + datetime format
#Date_rel_dt = pd.to_datetime(list_unduplicated['Date mise en relation'])
#Date_rel_dt.dt.date

from datetime import datetime

# create a 'date_without_time' column 
# list_unduplicated['date_without_time'] = list_unduplicated['Date mise en relation'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M').date())



# Apply lambda function to extract date part without time
list_unduplicated['date_without_time'] = list_unduplicated['Date mise en relation'].apply(lambda x: x.date())
# Convert the 'Date mise en relation' column to datetime objects
list_unduplicated['date_without_time'] = pd.to_datetime(list_unduplicated['date_without_time'])

type(list_unduplicated['date_without_time'])
list_unduplicated


In [ ]:
list_unduplicated.groupby('Antenne').count()

In [ ]:
#map empty/na cells
mno.matrix(list_unduplicated)

In [ ]:
#check if any null data
list_unduplicated.isnull().sum()


PARTNERS LIST TABLES

In [ ]:
#create partners table
df_partners = list_unduplicated[~list_unduplicated.duplicated(subset=['Partenaire'])].copy()
df_partners = df_partners[ ['Partenaire', 'Contact partenaire', 'Antenne'] ]
df_partners.sort_values(by='Partenaire', inplace=True)

df_partners.loc[(df_partners['Partenaire'].str.contains('Groupama'))]
#df_partners[groupama]

In [ ]:
#clean the datas
from unidecode import unidecode

# Define lambdas for transformations
lower_index = lambda x: x.lower() if isinstance(x, str) else x
erase_indent = lambda x: x.replace("-", " ") if isinstance(x, str) and '@' not in x else x
strip_index = lambda x: x.strip() if isinstance(x, str) else x
accents_strip = lambda x: unidecode(x) if isinstance(x, str) else x  # Apply unidecode only to string values
erase_apostrophe = lambda x: x.replace("'", " ") if isinstance(x, str) else x

# Identify columns with dtype 'object' (string columns)
str_columns_list = [str_column for str_column in df_partners.columns if df_partners.columns.dtype == 'object']

# Apply transformations to selected string columns
for str_column in str_columns_list:
    df_partners[str_column] = df_partners[str_column].apply(lower_index).apply(erase_indent).apply(strip_index).apply(accents_strip).apply(erase_apostrophe)

# Output the transformed DataFrame
df_partners

In [ ]:
#reset_index
df_partners.reset_index(drop=True)

------------ End of partners list


In [ ]:
df_partners.to_csv('list_parteners_02_2024.csv', index=False)

In [ ]:
#clean datas

## replace null by numpy "nan"
#list_unduplicated_unblanked = list_unduplicated.apply(lambda x: x.str.strip().replace('', np.nan))
list_unduplicated_unblanked = list_unduplicated.replace('', np.nan)

## count how many null
#list_unduplicated_unblanked.isnull().sum()
list_unduplicated_unblanked.isna().sum()



In [ ]:
#fillna blank cells
list_unduplicated_unblanked['Contact partenaire'].fillna('inconnu@adfinir.com', inplace=True)
list_unduplicated_unblanked['Telephone'].fillna('0600000000', inplace=True)
list_unduplicated_unblanked.isna().sum()



In [ ]:
#explore list and datas
list_unduplicated_unblanked.head()
list_unduplicated_unblanked.info()

In [ ]:
# check duplicated and select unduplicated only
list_duplicated_unblanked=list_unduplicated_unblanked.duplicated(subset=['Entrepreneur', 'Email', 'Telephone', 'Partenaire', 'Catégorie partenaire', 'Conseiller','Antenne', 'Date mise en relation', 'date_without_time'])
list_unduplicated_unblanked = list_unduplicated_unblanked[~list_duplicated_unblanked]
list_unduplicated_unblanked 

In [ ]:
#erase useless columns and reset index
list_cleaned = list_unduplicated_unblanked.drop(columns=['Telephone', 'BGE Club', 'Contact partenaire', 'Date mise en relation', 'A été contacté', 'A eu un échange', 'Note'] )
list_cleaned = list_cleaned.reset_index(drop=True)

In [ ]:
from unidecode import unidecode

# Define lambdas for transformations
lower_index = lambda x: x.lower() if isinstance(x, str) else x
erase_indent = lambda x: x.replace('-', ' ') if isinstance(x, str) else x
strip_index = lambda x: x.strip() if isinstance(x, str) else x
accents_strip = lambda x: unidecode(x) if isinstance(x, str) else x  # Apply unidecode only to string values
erase_apostrophe = lambda x: x.replace("'", " ") if isinstance(x, str) else x

# Define function to decode string using multiple encodings
def decode_string(original_string):
    if isinstance(original_string, int):  # Check if input is an integer (likely an Excel date)
        return original_string  # Return the integer as is
    elif isinstance(original_string, float):  # Check if input is a float
        return original_string  # Return the float as is
    else:
        encodings_to_try = ['utf-8', 'latin-1', 'windows-1252']
        encoded_string = original_string.encode('utf-8')  # Encode once before the loop
        for encoding in encodings_to_try:
            try:
                decoded_text = encoded_string.decode(encoding)
                return decoded_text
            except UnicodeDecodeError:
                print(f"Failed to decode with {encoding}")
        return original_string  # Return original value if all decoding attempts fail

# Identify columns with dtype 'object' (string columns)
str_columns_list = [str_column for str_column in list_cleaned.columns if list_cleaned[str_column].dtype == 'object']

# Apply transformations to selected string columns
for column in str_columns_list:
    list_cleaned[column] = list_cleaned[column].apply(decode_string).apply(lower_index).apply(erase_indent).apply(strip_index).apply(accents_strip).apply(erase_apostrophe)

# Output the transformed DataFrame
list_cleaned

In [ ]:
#create a csv file
list_cleaned.to_csv('list_cleaned_02_29v2.csv', index=False)